# Chapter 33- Beta rhythms from E-I Network
This notebook is a translation of an example in Christophe Borgers' book, "An Introduction to Modeling Neuronal Dynamics"

In this tutorial,we display an pyramidal - Interneuronal Network Gamma (PING) network entrain excitatory neurons (E-cells). Beta rhythms are 12-30 Hz, thus the firing of the E-cells must be within this range. We model 200 E-cells with an RTM model and the 50 PING cells with a WB model.

requirements:

 bmtk

 NEURON 7.4+

### Create nodes
The PING network, E-cells, and external input nodes are defined

In [1]:
! pip install -q h5py==2.9.0
! pip install -q bmtk==0.0.9
! pip install -q neuron==8.0.0

import h5py
import bmtk
import neuron
#print('The version of hfpy installed is',h5py.__version__)
# as new version of neuron and bmtk come out we may need to install a specific version
import os
from os.path import normpath, sep, join

root = 'Borgers-Examples-in-BMTK' # name of github repo
folder = '33-EI Beta' # name of folder in Github just change this line 
pathlist = normpath(os.getcwd()).split(sep)
if pathlist[-1] != folder:
  rootidx = pathlist.index(root) if root in pathlist else -1
  if rootidx>0:
    os.chdir(join(sep,*pathlist[:rootidx]))
  !git clone https://github.com/Kally367294/Borgers-Examples-in-BMTK
  os.chdir(join(root,folder))
print(os.getcwd())
%ls

     |████████████████████████████████| 2.8 MB 4.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
     |████████████████████████████████| 4.8 MB 4.4 MB/s 
     |████████████████████████████████| 12.6 MB 4.5 MB/s 
Cloning into 'Borgers-Examples-in-BMTK'...
remote: Enumerating objects: 874, done.
remote: Counting objects: 100% (874/874), done.
remote: Compressing objects: 100% (489/489), done.
remote: Total 874 (delta 462), reused 718 (delta 357), pack-reused 0
Receiving objects: 100% (874/874), 30.76 MiB | 10.21 MiB/s, done.
Resolving deltas: 100% (462/462), done.
/content/Borgers-Examples-in-BMTK/33-EI Beta
'33- EI Beta.ipynb'   components/   input/


In [2]:
%ls
%cd components/mechanisms/
!nrnivmodl
%cd ..
%cd ..
%ls

'33- EI Beta.ipynb'   components/   input/
/content/Borgers-Examples-in-BMTK/33-EI Beta/components/mechanisms
/content/Borgers-Examples-in-BMTK/33-EI Beta/components/mechanisms
Mod files: "./gap.mod" "./k_rtm.mod" "./k_wb.mod" "./leak.mod" "./na_rtm.mod" "./na_wb.mod" "./vecevent.mod"

Creating x86_64 directory for .o files.

COBJS=''
 -> NMODL ../k_wb.mod
(cd ".."; MODLUNIT=/usr/local/lib/python3.7/dist-packages/neuron/.data/share/nrn/lib/nrnunits.lib /usr/local/lib/python3.7/dist-packages/neuron/.data/bin/nocmodl k_wb.mod -o "/content/Borgers-Examples-in-BMTK/33-EI Beta/components/mechanisms/x86_64")
 -> NMODL ../k_rtm.mod
(cd ".."; MODLUNIT=/usr/local/lib/python3.7/dist-packages/neuron/.data/share/nrn/lib/nrnunits.lib /usr/local/lib/python3.7/dist-packages/neuron/.data/bin/nocmodl k_rtm.mod -o "/content/Borgers-Examples-in-BMTK/33-EI Beta/components/mechanisms/x86_64")
 -> NMODL ../gap.mod
(cd ".."; MODLUNIT=/usr/local/lib/python3.7/dist-packages/neuron/.data/share/nrn/lib/nrnunits.

In [3]:
import numpy as np
import logging
import math

from bmtk.builder.networks import NetworkBuilder

# E - Cells
n_E = 200

net = NetworkBuilder('network')
net.add_nodes(N=n_E, pop_name='Exc',     # N = number of excitatory cells
        model_type='biophysical',
        model_template='hoc:RTMExcCell', # RTMExcCell hoc definition
        morphology='blank.swc')

# ING network
n_I = 50

net.add_nodes(N=n_I, pop_name='PING',     # N = number of ING cells
        model_type='biophysical',
        model_template='hoc:WBInhCell', 
        morphology='blank.swc')

### Connection rules 

We add connections based on a probability defined below. To prevent autapses, if the node_id is equal between source and targe cell, no connection is made.

In [4]:
import random

def syn_connector(source,target,p):
    if source['node_id'] == target['node_id']:
        return 0
    return 1 if random.random() < p else 0

### Creating ING Edges

Probabilities and conductances that are commented are taken from chapter 33 of Borgers' book. Inhibitory connections between PING cells are added; Inhibitory connections from PING cells to E - cells are added; Excitatory connections between E-cells and PING cells are added.

In [5]:
g_ee = 0
g_ei = 0.1  #0.25
g_ie = 0.5  #0.5
g_ii = 0.5  #0.5

p_ee = 0.5  # 0.5
p_ei = 0.5  # 0.5
p_ie = 0.5  # 0.5
p_ii = 1   # 0.5



#E-E connections dont exist (g_ee = 0)
         
#E-I connections 
net.add_edges(source={'pop_name':'Exc'}, target={'pop_name':'PING'},
                    connection_rule=syn_connector, 
                    connection_params={'p':p_ei},
                    delay=0,
                    syn_weight = g_ei/n_E,
                    weight_function=None,
                    target_sections=['soma'],
                    distance_range=[0.0, 0.1],
                    dynamics_params='PING_ExcToInh.json',
                    model_template='Exp2Syn')

#I-I connections
net.add_edges(source={'pop_name':'PING'}, target={'pop_name':'PING'},
                    connection_rule=syn_connector, 
                    connection_params={'p':p_ii},
                    delay=0,
                    syn_weight = g_ii,
                    weight_function=None,
                    target_sections=['soma'],
                    distance_range=[0.0, 0.1],
                    dynamics_params='PING_InhToInh.json',
                    model_template='Exp2Syn')
                    
#I-E connections
net.add_edges(source={'pop_name':'PING'}, target={'pop_name':'Exc'},
                    connection_rule=syn_connector,
                    connection_params={'p':p_ie},
                    delay=0,
                    syn_weight = g_ie,
                    weight_function=None,
                    target_sections=['soma'],
                    distance_range=[0.0, 0.1],
                    dynamics_params='PING_InhToExc.json',
                    model_template='Exp2Syn')


In [6]:
net.build()
net.save_nodes(output_dir='network')
net.save_edges(output_dir='network')

### Current Clamps for E-Cells and ING network

In [ ]:
import h5py

I_bar_E = 1.5      
sigma_E = 0.1   
I_bar_I = 1.5      
sigma_I = 0.1   


n_I = 50
n_E = 200
sim_len = 400
dt = 1
samples = int(sim_len/dt)
amp = np.zeros((n_E+n_I,samples))


amp[n_I:,:] = (I_bar_E * (1 + sigma_E*np.random.normal(size=n_E))).reshape(-1,1) # Excitatory Cell Current Injection
amp[:n_I,:] = (I_bar_I * (1 + sigma_I*np.random.normal(size=n_I))).reshape(-1,1) # Inhibitory Cell Current Injection


dts = [1]
gids = "all"

#hf.close() # Uncomment this line temporarily if you receive 'file already open error'

hf = h5py.File("input/amps.h5", 'w')

hf.create_dataset("amplitudes", data=[amp])
hf.create_dataset("gids", data=gids)
hf.create_dataset("dts", data=dts)

hf.close()

In [8]:
%ls

'33- EI Beta.ipynb'   components/   input/   network/


### Set up Simulation Environment

In [9]:
from bmtk.utils.sim_setup import build_env_bionet

build_env_bionet(base_dir='.',
                 network_dir='network',
                 tstop=400.0, dt=0.1,
                 report_vars=['v'],           # Record membrane potential and calcium (default soma)
                 #spikes_inputs=[('extnet',   # Name of population which spikes will be generated for
                                #'input/ext_inp_poisson.h5')],
                 #file_current_clamp={              
                      #"input_type": "file_current_clamp",
                      #"module": "FileIClamp",
                      #"input_file":"input/amps.h5",
                      #"node_set":"all"
                # },
                 current_clamp={            # Creates a step current from 500.ms to 1500.0 ms
                    'amp': 0.400,
                    'delay': 0,
                    'duration': 30.0
                },
                 v_init= -65,
                 include_examples=False,    # Copies components files
                 compile_mechanisms=False,   # Will try to compile NEURON mechanisms
                 config_file = 'config.json'
                )

### Run the Simulation

In [ ]:
from bmtk.simulator import bionet


conf = bionet.Config.from_json('config.json')
conf.build_env()
net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()

2022-04-12 22:51:17,275 [INFO] Created log file


INFO:NEURONIOUtils:Created log file


2022-04-12 22:51:17,379 [INFO] Building cells.


INFO:NEURONIOUtils:Building cells.


2022-04-12 22:51:17,636 [INFO] Building recurrent connections


INFO:NEURONIOUtils:Building recurrent connections


### Analyze Results

We did not add initial voltage values to the PING network nor the E - cells. Because of a synchronous initialization, only the ING cell entrainment of the E-cells can be observed. The E - cells are already synchronous.

In [ ]:
from bmtk.analyzer.spike_trains import plot_raster,plot_rates_boxplot

plot_raster(config_file='config.json', with_histogram=False, population='network', group_by='pop_name')

In [ ]:
plot_rates_boxplot(config_file='config.json', group_by='pop_name')

The PING network successfully entrained the E-cells (Exc) at ~20 Hz which is in Beta wave range